In [1]:
!pip install streamlit

     |████████████████████████████████| 7.2MB 1.6MB/s 
     |████████████████████████████████| 112kB 35.7MB/s 
     |████████████████████████████████| 102kB 9.6MB/s 
     |████████████████████████████████| 4.4MB 32.3MB/s 
     |████████████████████████████████| 122kB 40.5MB/s 
  Created wheel for blinker: filename=blinker-1.4-cp36-none-any.whl size=13450 sha256=308341438807dfb4b5e8f25aa19c2b94edbb51d712c50c4d2b136297b32a547a
  Stored in directory: /root/.cache/pip/wheels/92/a0/00/8690a57883956a301d91cf4ec999cc0b258b01e3f548f86e89
  Created wheel for watchdog: filename=watchdog-0.10.3-cp36-none-any.whl size=73873 sha256=6a1cd0fc49503d7534ee222aa0c881bd5332f4209cd47a858715b312f1d956b1
  Stored in directory: /root/.cache/pip/wheels/a8/1d/38/2c19bb311f67cc7b4d07a2ec5ea36ab1a0a0ea50db994a5bc7
  Created wheel for pathtools: filename=pathtools-0.1.2-cp36-none-any.whl size=8785 sha256=db5798d2a62fe16361249a64303b516fe1b3c24caefcbd33d33de629bc773afe
  Stored in directory: /root/.cache/pip/wheel

In [1]:
pip install adversarial-robustness-toolbox

     |████████████████████████████████| 655kB 1.9MB/s 
     |████████████████████████████████| 20.8MB 1.5MB/s 
     |████████████████████████████████| 6.8MB 42.5MB/s 
     |████████████████████████████████| 235kB 39.3MB/s 
     |████████████████████████████████| 747kB 27.2MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [2]:
!pip install pyngrok

  Created wheel for pyngrok: filename=pyngrok-4.1.12-cp36-none-any.whl size=16810 sha256=61a70233da7c1fdd2b66dc89267cd545ef5fc0c3ef6edf9f98570762d04d6254
  Stored in directory: /root/.cache/pip/wheels/69/c3/d6/6968dd4d831794d41c311be1d7af6f4ac151c5d3bd0e6efab8
Successfully built pyngrok


In [3]:
!wget https://raw.githubusercontent.com/TanmayKhot/Fluorine/master/imagenet_classes.txt

--2020-09-17 06:34:10--  https://raw.githubusercontent.com/TanmayKhot/Fluorine/master/imagenet_classes.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21675 (21K) [text/plain]
Saving to: ‘imagenet_classes.txt’

imagenet_classes.tx 100%[===================>]  21.17K  --.-KB/s    in 0.02s   

2020-09-17 06:34:10 (1.04 MB/s) - ‘imagenet_classes.txt’ saved [21675/21675]



In [40]:
%%writefile main.py
import streamlit as st
PAGE_CONFIG = {"page_title":"Image Classifier Visualizer with Adv Attacks.io",  "layout":"centered"}
st.beta_set_page_config(**PAGE_CONFIG)
import numpy as np
import time
import torch
import torchvision.models as models
from torchvision import transforms as T
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from PIL import Image
import urllib.request
from art.attacks.evasion import *
from art.estimators.classification import PyTorchClassifier

with open('imagenet_classes.txt') as f:
  classes = [line.strip() for line in f.readlines()]
st.set_option('deprecation.showfileUploaderEncoding', False)
html_temp = '''
    <div style = "background-color: rgba(25,25,112,0.20); padding: 5px; padding-left: 5px; padding-right: 5px">
    <center><h1>Image Classifier</h1></center>
    <center><h2>with</h2></center>
    <center><h1>Adversarial Attack</h1></center>
    <h5></h5>
    </div>
    '''
st.markdown(html_temp, unsafe_allow_html=True)

def paginator(label, items, items_per_page=2, on_sidebar=True):
    max_index =  items_per_page
    import itertools
    return itertools.islice(enumerate(items), max_index)

def classify(model, image, classes, model_select, choice):
  model.eval()
  from torchvision import transforms
  
  preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
  ])
  input_tensor = preprocess(image)
  input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model
 
  # move the input and model to GPU for speed if available
  if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')
  with torch.no_grad():
    output = model(input_batch)
  _, index = torch.max(output, 1)
  percentage = torch.nn.functional.softmax(output, dim=1)[0] * 100
  
  st.success('Hey! {} classified your image as a {} with confidence {:.3f}'.format(model_select, classes[index[0]], percentage[index[0]].item()))
  
def adversarial(image, model, attack, attack_select, epsilon):
  width, height = image.size
  transform = T.Compose([            
    T.Resize(256),                    
    T.CenterCrop(224),                
    T.ToTensor(),                     
    T.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225])
  ])
  
  transformed_img = transform(image)
  input_img = transformed_img.unsqueeze(0)
  advx = attack.generate(x=input_img)
  rescaled = (advx - advx.min()) / (advx.max() - advx.min())
  rescaled = (255*np.transpose(np.squeeze(rescaled))).astype(np.uint8)
  rescaled = np.moveaxis(rescaled ,0, 1)
  img = Image.fromarray(rescaled)
  f_trans = T.Compose([
                       T.Resize(256),
                       T.CenterCrop(224)])
  img_f = f_trans(image)
  images=[img_f, img]
  captions=['Uploaded Image', 'Image after Adversarial Attack']
  image_iter = paginator("",images)
  index, fimg = map(list, zip(*image_iter))
  st.image(fimg, width = 328, caption = captions)
  transformed_img = transform(img)
  final_img = transformed_img.unsqueeze(0)
  output = model(final_img)
  _, index = torch.max(output, 1)
  percentage = torch.nn.functional.softmax(output, dim=1)[0] * 100
  st.warning('Hey! {} classified your image as a {} with confidence {:.3f} after implementing {}'.format(model_select, classes[index[0]], percentage[index[0]].item(), attack_select))

html_temp = '''
    <div>
    <h2></h2>
    <center><h3>Please upload Image for Classification</h3></center>
    </div>
    '''
st.markdown(html_temp, unsafe_allow_html=True)
opt = st.selectbox("How do you want to upload the image for classification?\n", ('Please Select', 'Upload image via link', 'Upload image from device'))
if opt == 'Upload image from device':
    file = st.file_uploader('Select', type = ['jpg', 'png', 'jpeg'])
    st.set_option('deprecation.showfileUploaderEncoding', False)
    if file is not None:
        image = Image.open(file)

elif opt == 'Upload image via link':
    img = st.text_input('Enter the Image Address')
    image = Image.open(urllib.request.urlopen(img))
model_select = st.selectbox('Please select a Classification Model from the list',('','AlexNet', 'DenseNet', 'GoogLeNet', 'InceptionNet', 'ResNet','VGG'))
attack_select = st.selectbox('Please select an Adversarial Attack from the list:',('None','DeepFool','FGSM','PGD','AutoAttack','BasicIterativeMethod')) 
if attack_select != 'None':
      epsilon = st.number_input('Enter Epsilon value(e>0):')
      if epsilon < 0:
        warn = st.error("Epsilon value should be a positive number. Please Re-enter...")
        time.sleep(3)
        warn.empty()

if st.button('Classify'): 


  if attack_select == 'None':
    try:
      if model_select == 'VGG':
         model = models.vgg16(pretrained=True)
         st.image(image, width = 300, caption = 'Uploaded Image')
         classify(model, image, classes, model_select, 1)
      elif model_select == 'AlexNet':
         model = models.alexnet(pretrained=True)
         st.image(image, width = 300, caption = 'Uploaded Image')
         classify(model, image, classes, model_select, 1)
      elif model_select == 'DenseNet':
         model = models.densenet121(pretrained=True)
         st.image(image, width = 300, caption = 'Uploaded Image')
         classify(model, image, classes, model_select, 1)
      elif model_select == 'ResNet':
         model = models.resnet101(pretrained=True)
         st.image(image, width = 300, caption = 'Uploaded Image')
         classify(model, image, classes, model_select, 1)
      elif model_select == 'GoogLeNet':
         model = models.googlenet(pretrained=True)
         st.image(image, width = 300, caption = 'Uploaded Image')
         classify(model, image, classes, model_select, 1)
      elif model_select == 'InceptionNet':
         model = models.inception_v3(pretrained=True)
         st.image(image, width = 300, caption = 'Uploaded Image')
         classify(model, image, classes, model_select, 1)
    except:
      pass

  else:
   if model_select == 'VGG':
       model = models.vgg16(pretrained=True)
       
   elif model_select == 'AlexNet':
        model = models.alexnet(pretrained=True)
        
   elif model_select == 'DenseNet':
        model = models.densenet121(pretrained=True)
        
   elif model_select == 'ResNet':
        model = models.resnet101(pretrained=True)

   elif model_select == 'GoogLeNet':
         model = models.googlenet(pretrained=True)

   elif model_select == 'InceptionNet':
         model = models.inception_v3(pretrained=True)

   if model_select != '':
      criterion = nn.CrossEntropyLoss()
      optimizer = optim.Adam(model.parameters(), lr=0.01)
      classifier = PyTorchClassifier(
        model=model,
        clip_values=(0, 1),
        loss=criterion,
        optimizer=optimizer,
        input_shape=(3, 224, 224),
        nb_classes=1000,
      )
   try:
      if attack_select == 'DeepFool':
        attack = DeepFool(classifier, epsilon=epsilon)
        adversarial(image, model, attack,  attack_select, epsilon)
      elif attack_select == 'FGSM':
        attack = FastGradientMethod(estimator = classifier, eps = epsilon) 
        adversarial(image, model, attack,  attack_select, epsilon)
      elif attack_select == 'PGD':
        attack = ProjectedGradientDescent(classifier, eps = epsilon)
        adversarial(image, model, attack,  attack_select, epsilon)
      elif attack_select == 'AutoAttack':
        attack = AutoAttack(estimator = classifier, eps = epsilon)
        adversarial(image, model, attack,  attack_select, epsilon)
      elif attack_select == 'BasicIterativeMethod':
        attack = BasicIterativeMethod(classifier, epsilon)
        adversarial(image, model, attack,  attack_select, epsilon)
   except:
      pass

   if opt == "Please Select":
        pass






Overwriting main.py


In [5]:
!ngrok authtoken 1hQAE2uizes35CU6alr9SoHkWrn_5hci3LaKPin2vFJcvLhks

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [6]:
!streamlit run main.py&>/dev/null&
!pgrep streamlit

426


In [7]:
from pyngrok import ngrok

In [8]:
public_url = ngrok.connect(port='8501')
public_url

'http://8565e95432ef.ngrok.io'

In [41]:
ngrok.kill()